In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [ ]:
import pandas as pd
from google.colab import files
uploaded = files.upload()

df = pd.read_csv('tmdb_5000_movies.csv')

df['text'] = df['title'].fillna('') + '. ' + df['overview'].fillna('')




In [ ]:
texts = df['text'].tolist()[:500]
encoded_inputs = tokenizer(texts, padding=True, truncation=True, max_length=512, return_tensors="pt")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("w11wo/indonesian-roberta-base-sentiment-classifier")
model = AutoModelForSequenceClassification.from_pretrained("w11wo/indonesian-roberta-base-sentiment-classifier")

In [ ]:
encoded_inputs = tokenizer(
    texts,
    padding=True,
    truncation=True,
    max_length=512,
    return_tensors="pt",
    return_attention_mask=True
)

In [ ]:
model.eval()
predictions = []
batch_size = 8

for i in range(0, len(texts), batch_size):
    batch_texts = texts[i : i + batch_size]
    batch_enc = tokenizer(
        batch_texts,
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors="pt",
        return_attention_mask=True
    )
    with torch.no_grad():
        outputs = model(**batch_enc)
        probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
        predictions.append(probs)


predictions = torch.cat(predictions, dim=0)

In [ ]:
for i, txt in enumerate(texts[:10]):
    label = "positif" if predictions[i,1] > predictions[i,0] else "negatif"
    confidence = float(predictions[i].max())
    print(f"Movie {i+1}: {df.loc[i,'title']}")
    print(f"Text Preview: {txt[:100]}...")
    print(f"Sentimen: {label} (Confidence: {confidence:.4f})\n")

Movie 1: Avatar
Text Preview: Avatar. In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique missi...
Sentimen: positif (Confidence: 0.4751)

Movie 2: Pirates of the Caribbean: At World's End
Text Preview: Pirates of the Caribbean: At World's End. Captain Barbossa, long believed to be dead, has come back ...
Sentimen: negatif (Confidence: 0.9306)

Movie 3: Spectre
Text Preview: Spectre. A cryptic message from Bond’s past sends him on a trail to uncover a sinister organization....
Sentimen: negatif (Confidence: 0.5847)

Movie 4: The Dark Knight Rises
Text Preview: The Dark Knight Rises. Following the death of District Attorney Harvey Dent, Batman assumes responsi...
Sentimen: negatif (Confidence: 0.7171)

Movie 5: John Carter
Text Preview: John Carter. John Carter is a war-weary, former military captain who's inexplicably transported to t...
Sentimen: negatif (Confidence: 0.7134)

Movie 6: Spider-Man 3
Text Preview: Spider-Man 3. The seemingly invincible 